# Multi-Agent Financial Analysis System

This notebook implements a multi-agent system using **Agno** framework with **Google Gemini** models.

## Agents
1. **Financial Document Knowledge Base Expert** - Handles financial sentiment analysis from news
2. **Real-Time CSV Data Agent** - Processes stock price and volume data
3. **Team Leader** - Coordinates responses from both agents

## System Architecture
```
User Query
    ↓
Team Leader Agent (Orchestrator)
    ↓
    ├──→ Financial Document Expert Agent (Gemini 2.0 Flash)
    └──→ CSV Data Agent (Gemini 2.0 Flash)
    ↓
Final Integrated Response
```

## Data Sources
- **Finance Agent**: `financeAgent/data/` - Financial sentiment analysis data
- **CSV Agent**: `csvAgent/data/` - Stock price and volume data


In [1]:
# Install required packages from requirements.txt
import subprocess
import sys

print("Installing required packages from requirements.txt...")
try:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt", "-q"])

    print("✓ All packages installed successfully!")
except subprocess.CalledProcessError as e:
    print(f"✗ Failed to install packages: {e}")

print("\nInstallation complete!")


Installing required packages from requirements.txt...
✓ All packages installed successfully!

Installation complete!



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
# Import required libraries
import os
import glob
import pathlib
from agno.knowledge.text import TextKnowledgeBase
from agno.knowledge.csv import CSVKnowledgeBase
from agno.agent import Agent

In [3]:
finance_data_dir = pathlib.Path("financeAgent/data")
csv_data_dir = pathlib.Path("csvAgent/data")

finance_data_dir.mkdir(parents=True, exist_ok=True)
csv_data_dir.mkdir(parents=True, exist_ok=True)

In [4]:
finance_kb = TextKnowledgeBase(
    name="finance_kb",
    description="Financial sentiment and document knowledge base",
)

# Path to the directory containing text files
finance_data_dir = "path/to/finance_data_dir"

# Iterate over all .txt files in the directory
for file_path in glob.glob(os.path.join(finance_data_dir, "*.txt")):
    with open(file_path, "r", encoding="utf-8") as file:
        content = file.read()  # Read the content of the file
        finance_kb.add_document(content) 

In [5]:
# Create a new knowledge base for CSV documents
csv_kb = CSVKnowledgeBase(
    name="csv_kb",
    description="Financial CSV knowledge base from CSV documents",
    path="path/to/store/csv_kb"  # Specify the path for the knowledge base
)

# Path to the directory containing CSV files
csv_data_dir = "path/to/finance_csv_data_dir"

# Iterate over all .csv files in the directory
for file_path in glob.glob(os.path.join(csv_data_dir, "*.csv")):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    # Convert the DataFrame to a string (or process it as needed)
    content = df.to_string(index=False)  # Convert to a string without row indices
    csv_kb.add_document(content)

In [6]:

from agno.models.google import Gemini

# Define a subclass of Gemini
class CustomGemini(Gemini):
    def parse_provider_response(self, response):
        # Implement the logic to parse the provider's response
        # Replace this with the actual parsing logic
        return {"parsed_response": response}

    def parse_provider_response_delta(self, response_delta):
        # Implement the logic to parse the provider's response delta
        # Replace this with the actual parsing logic
        return {"parsed_response_delta": response_delta}

In [7]:
from agno.models.google import Gemini
finance_agent = Agent(
    name="Finance_Document_Expert",
    role="Financial Document Knowledge Base Specialist",
    model=Gemini(id="gemini-2.5-pro"),
    tools=[finance_kb],  # Add the knowledge base as a tool
    instructions=[
        "Analyze financial documents thoroughly and provide detailed insights",
        "Reference specific sections of documents when making claims",
        "Provide context and explanations for financial terms and concepts",
        "Be precise with numbers, dates, and financial calculations",
        "When uncertain, clearly state what information is missing",
    ],
    markdown=True,
)

print("✓ Finance Document Expert Agent created successfully!")

✓ Finance Document Expert Agent created successfully!


In [8]:
csv_agent = Agent(
    name="CSV_Data_Analyst",
    role="Real-Time CSV Data Analyst",
    model=Gemini(id="gemini-2.5-pro"),  # Use the subclass here
    tools=[csv_kb],
    instructions=[
        "Analyze CSV data to extract relevant insights based on queries",
        "Provide statistical summaries and key findings",
        "Highlight trends and patterns in the data",
        "Be specific with numbers and percentages",
        "Mention data quality limitations if present"
    ],
    markdown=True,
)

print("✓ CSV Data Agent created successfully!")


✓ CSV Data Agent created successfully!


In [9]:
# Create Team Leader Agent
team_leader = Agent(
    name="Team_Leader",
    role="Multi-Agent Coordinator and Team Leader",
    model=Gemini(id="gemini-2.5-pro"),
    team=[finance_agent, csv_agent],
    instructions=[
        "Understand the user's query and determine what information is needed",
        "Coordinate with both Finance and CSV agents to gather insights",
        "Synthesize responses from multiple agents into a coherent answer",
        "Highlight synergies and contradictions between document knowledge and real data",
        "Provide a balanced perspective that combines both sources",
        "Clearly structure the final response with sections and proper formatting",
        "If agents provide conflicting information, present both views with context"
    ],
    tools=[finance_agent, csv_agent],  # Add the two agents here
    markdown=True,
)

print("✓ Team Leader Agent created")


✓ Team Leader Agent created


In [10]:
# %pip install agno==1.5.9
def ask_team(query: str):
    try:
        result = team_leader.run(query)
        return result.content
    except Exception as e:
        return f"Error processing query: {str(e)}"


## Usage Examples

The system is now ready to use! Below are some example queries you can try.


In [11]:
# Example 1: Basic query without CSV data
query1 = "What are the key principles of financial risk management?"

print("Query:", query1)
print("\n" + "="*80 + "\n")

response1 = ask_team(query1)


Query: What are the key principles of financial risk management?




In [14]:
print(response1)

Of course. Here is a detailed explanation of the core principles of financial risk management, based on established financial doctrines.

### The Core Principles of Financial Risk Management

Financial Risk Management is a continuous, iterative process designed to identify, assess, control, and monitor the financial risks an organization faces. The goal is not to eliminate all risk—as risk is inherent in seeking returns—but to manage it effectively to protect and enhance organizational value.

The process is generally broken down into four key principles that form a continuous cycle:

1.  **Risk Identification**
2.  **Risk Assessment & Measurement**
3.  **Risk Mitigation & Response**
4.  **Risk Monitoring & Reporting**

---

### 1. Risk Identification: "What can go wrong?"

This is the foundational step. An organization cannot manage a risk it is unaware of. The goal is to create a comprehensive inventory of all potential financial risks that could adversely affect the company's earnin

## Summary

Your multi-agent financial analysis system is now complete! 

### System Overview
- **3 Specialized Agents**: Each agent uses Google Gemini 2.0 Flash model
- **Coordinated Responses**: The Team Leader orchestrates answers from both data sources
- **Local Data Support**: CSV Agent processes data from `csvAgent/data/`
- **Sentiment Analysis**: Finance Agent analyzes financial sentiment from `financeAgent/data/`

### Next Steps
1. Set your `GOOGLE_API_KEY` in cell 3
2. Run the data loading cell (Example 2) to load your datasets
3. Start asking questions to the multi-agent system!

### Tips
- The system works best when combining insights from both agents
- CSV data is automatically analyzed for trends and statistics
- Finance data provides sentiment analysis capabilities
- All responses are formatted in Markdown for readability
- The Team Leader ensures comprehensive, well-structured answers
